In [1]:
# Necessary imports
import nltk
import numpy as np
import pandas as pd
import tensorflow as ts
import matplotlib.pyplot as plt

In [2]:
# Reading in the positive voted comments and removing NaN rows
df_pos_full = pd.read_csv('comments_positive.csv')
print('Originial', df_pos_full.shape)
df_pos_full.dropna(axis=0, inplace=True)
print('NaN removed',df_pos_full.shape)

# Reading in the negative voted comments and removing NaN rows
df_neg_full = pd.read_csv('comments_negative.csv')
print('Originial', df_neg_full.shape)
df_neg_full.dropna(axis=0, inplace=True)
print('NaN removed',df_neg_full.shape)

Originial (2000000, 15)
NaN removed (1999977, 15)
Originial (2000000, 15)
NaN removed (1999951, 15)


In [3]:
df_pos_full.head()

,id,parent_id,subreddit_id,link_id,text,score,ups,author,controversiality,parent_link_id,parent_text,parent_score,parent_ups,parent_author,parent_controversiality
0,c092j8m,t1_c092gss,t5_2qh2p,t3_8eyy3,This isn't Twitter: try to comment on the arti...,9582,9582,nraustinii,0,t3_8eyy3,Fucking faggot.,-7526,-7526,Glorificus,0
1,c4imcva,t1_c4im948,t5_2qh1i,t3_t0ynr,"Well, it is exactly what it sounds like. It's ...",9531,9531,Lynfect,0,t3_t0ynr,"Elaborate on this cum box, please.",3841,3841,eeeeevil,0
2,c0s4nfi,t1_c0s4lje,t5_2qh1i,t3_cf1n2,"In soviet Russia, bomb disarms you!",8545,8545,CapnScumbone,0,t3_cf1n2,"I don't live in Russia anymore, and I will not...",621,621,shady8x,0
3,c4ini33,t1_c4incln,t5_2qh1i,t3_t0ynr,"""runin for senitur! #YOLO!""",7430,7430,[deleted],0,t3_t0ynr,This just made me realize that future presiden...,4651,4651,drspg99,0
4,c4imgel,t1_c4ima2e,t5_2qh1i,t3_t0ynr,You step motherfucker.,7173,7173,jbg89,0,t3_t0ynr,I have sex with my step mom when my dad isn't ...,4251,4251,audir8,0


In [4]:
# df_neg_full.head()

In [5]:
# Our dataset consists of two times 2 million comments (rows) so we split it for easier experimenting. 
# 1% of 2 million = 20000
# df_pos = df_pos_full.sample(frac=0.025,random_state=200)
# df_neg = df_neg_full.sample(frac=0.025,random_state=200)
# df_pos = df_pos.reset_index(drop=True)
# df_neg = df_neg.reset_index(drop=True)

# print(df_pos.shape)
# print(df_neg.shape)

We're splitting the data in 200 seperate pieces with each 10000 entries

In [6]:
df_pos_split = np.array_split(df_pos_full, 200)
df_neg_split = np.array_split(df_neg_full, 200)

As we can see in our dataset, the most important columns for our language processing are `text` and `parent_text`

So let's get some statistics about these columns!

In [7]:
# num_words_pos = df_pos['text'].apply(lambda x: len(x.split()))
# num_words_neg = df_neg['text'].apply(lambda x: len(x.split()))

# pos_words_mean, pos_words_std = np.mean(num_words_pos), np.std(num_words_pos)
# neg_words_mean, neg_words_std = np.mean(num_words_neg), np.std(num_words_neg)

# print("Positive stats:", pos_words_mean, pos_words_std)
# print("Negative stats:", neg_words_mean, neg_words_std)

In [8]:
def clean(text, stemming=False, stop_words=True):
    import re
    from string import punctuation
    from nltk.stem import SnowballStemmer
    from nltk.corpus import stopwords
    from nltk import word_tokenize
    
    stops = stopwords.words('english')
    
    # Empty comment
    if type(text) != str or text=='':
        return ''
    
    # Commence the cleaning!
    urls = r'http(s)*:\/\/(\w|\.)+(\/\w+)*'
    text = re.sub(urls, '', text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are", text)
    text = re.sub("\'ve", " have", text)
    text = re.sub("\'d", " would", text)
    text = re.sub("cant", "can not", text)
    text = re.sub("can\'t", "can not", text)
    text = re.sub("isn\'t", "is not", text)
    text = re.sub("isnt", "is not", text)
    text = re.sub("whats", "what is", text)
    text = re.sub("what\'s", "what is", text)
    text = re.sub("shouldn't", "should not", text, flags=re.IGNORECASE)
    text = re.sub("I'm", "I am", text)
    text = re.sub(":", " ", text)
    # The comments contain \n for line breaks, we need to remove those too
    text = re.sub("\\n", " ", text)
    
    # Special characters
    text = re.sub('\&', " and ", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    
    # Remove punctuation
    text = ''.join([word for word in text if word not in punctuation]).lower()
    
    # If we want to do stemming...
    if stemming:
        sno = SnowballStemmer('english')
        text = ''.join([sno.stem[word] for word in text])
    
    # If we want to remove stop words...
    if stop_words:
        text = text.split()
        text = [word for word in text if word not in stops]
        text = ' '.join(text)
    
    return text
    

In [9]:
# Making it a binary classification with either positive or negative score
def posneg(number):
    if number > 0:
        return 1
    else:
        return 0

In [12]:
from pathlib import Path
from profanity_check import predict as prof_predict
from profanity_check import predict_prob
from textblob import TextBlob

for n in range(200):
    
    # Clean the text
    df_pos_split[n]['text'] = df_pos_split[n]['text'].apply(clean)
    df_pos_split[n]['parent_text'] = df_pos_split[n]['parent_text'].apply(clean)
    df_neg_split[n]['text'] = df_neg_split[n]['text'].apply(clean)
    df_neg_split[n]['parent_text'] = df_neg_split[n]['parent_text'].apply(clean)
    
    df_pos_split[n]['score'] = df_pos_split[n]['score'].apply(posneg)
    df_pos_split[n]['parent_score'] = df_pos_split[n]['parent_score'].apply(posneg)
    df_pos_split[n]['sentiment'] = [TextBlob(text).sentiment.polarity for text in df_pos_split[n]['text'].astype(str)]
    df_pos_split[n]['profanity'] = [prof_predict([text]) for text in df_pos_split[n]['text'].astype(str)]
    df_pos_split[n]['profanity_prob'] = [predict_prob([text]) for text in df_pos_split[n]['text'].astype(str)]
    df_pos_split[n]['profanity'] = df_pos_split[n]['profanity'].astype(int)
    df_pos_split[n]['profanity_prob'] = df_pos_split[n]['profanity_prob'].astype(float)
    
    df_neg_split[n]['score'] = df_neg_split[n]['score'].apply(posneg)
    df_neg_split[n]['parent_score'] = df_neg_split[n]['parent_score'].apply(posneg)
    df_neg_split[n]['sentiment'] = [TextBlob(text).sentiment.polarity for text in df_neg_split[n]['text'].astype(str)]
    df_neg_split[n]['profanity'] = [prof_predict([text]) for text in df_neg_split[n]['text'].astype(str)]
    df_neg_split[n]['profanity_prob'] = [predict_prob([text]) for text in df_neg_split[n]['text'].astype(str)]
    df_neg_split[n]['profanity'] = df_neg_split[n]['profanity'].astype(int)
    df_neg_split[n]['profanity_prob'] = df_neg_split[n]['profanity_prob'].astype(float)

    df_pos_split[n].dropna(axis=0, inplace=True)
    df_neg_split[n].dropna(axis=0, inplace=True)
    
    # We don't need all columns
    col_write = ['text', 'score', 'ups', 'controversiality', 'parent_text', 'parent_score', 'parent_ups', 'parent_controversiality', 'sentiment', 'profanity', 'profanity_prob']
    # Save everything in the 'data' folder
    p = Path('data/')
    number = str(n)
    df_pos_split[n].to_csv(Path(p, 'clean_positive_train_' + number + '.csv'), columns=col_write, index=False)
    df_neg_split[n].to_csv(Path(p, 'clean_negative_train_' + number + '.csv'), columns=col_write, index=False)

Now we have 200 clean files for both positive and negative (so 400 total) which we can slowly feed to our neural network to get better and better